In [1]:
import pandas as pd
import collections
import statsmodels.formula.api as smf

In [2]:
df = pd.read_csv('data/listings_full.csv')


## Split the amenities column into single columns

In [3]:
def convert_to_dict(x):
    x = x.replace('{','').replace('}','')
    x = x.replace('\"','').replace('\"','')
    x = x.split(',')
    return collections.Counter(x)

In [4]:
df.amenities = df.amenities.apply(convert_to_dict)

In [5]:
df_amenities = df.amenities.apply(pd.Series)
df_amenities.columns = ['amenities_'+col.replace(" ","") for col in df_amenities.columns]

In [6]:
df = pd.concat([df,df_amenities],axis=1)

In [7]:
df.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,amenities_Washer/Dryer,amenities_Wheelchairaccessible,amenities_Wideclearancetobed,amenities_Wideclearancetoshowerandtoilet,amenities_Widedoorway,amenities_Widehallwayclearance,amenities_Windowguards,amenities_WirelessInternet,amenities_translationmissing:en.hosting_amenity_49,amenities_translationmissing:en.hosting_amenity_50
0,17260587,https://www.airbnb.com/rooms/17260587,20170507222235,2017-05-08,Kunterbuntes Zimmer mit eigenem Bad für jedermann,"Meine Unterkunft ist gut für paare, alleinreis...",NaN,"Meine Unterkunft ist gut für paare, alleinreis...",none,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
1,17227881,https://www.airbnb.com/rooms/17227881,20170507222235,2017-05-08,Modernes Zimmer in Berlin Pankow,Es ist ein schönes gepflegtes und modernes Zim...,Das Haus befindet sich direkt vor eine Tram Ha...,Es ist ein schönes gepflegtes und modernes Zim...,none,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
2,1153584,https://www.airbnb.com/rooms/1153584,20170507222235,2017-05-08,Gästezimmer Berlin-Pankow,Unser Gästezimmer befindet sich im Dachgeschos...,"Wenn Ihr eine anspruchsvolles Ambiente sucht, ...",Unser Gästezimmer befindet sich im Dachgeschos...,none,"Unweit der Berliner City, im nördlichen Pankow...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
3,7832750,https://www.airbnb.com/rooms/7832750,20170507222235,2017-05-08,Sonniges Doppelzimmer+Nice Price!,Welcome! Hier vermiete ich ein kleines Wohlfüh...,Die Wohnung ist durch Ihre Lage sehr schön hel...,Welcome! Hier vermiete ich ein kleines Wohlfüh...,none,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0
4,11634962,https://www.airbnb.com/rooms/11634962,20170507222235,2017-05-08,Room for womens in Pankow 30 min from the city,Gemütliches Zimmer im ruhigen Teil von Berlin....,"Es handelt sich um ein großes, gemütliches Zim...",Gemütliches Zimmer im ruhigen Teil von Berlin....,none,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [8]:
df.columns

Index([u'id', u'listing_url', u'scrape_id', u'last_scraped', u'name',
       u'summary', u'space', u'description', u'experiences_offered',
       u'neighborhood_overview',
       ...
       u'amenities_Washer/Dryer', u'amenities_Wheelchairaccessible',
       u'amenities_Wideclearancetobed',
       u'amenities_Wideclearancetoshowerandtoilet', u'amenities_Widedoorway',
       u'amenities_Widehallwayclearance', u'amenities_Windowguards',
       u'amenities_WirelessInternet',
       u'amenities_translationmissing:en.hosting_amenity_49',
       u'amenities_translationmissing:en.hosting_amenity_50'],
      dtype='object', length=199)

In [15]:
[col for col  in df.columns.tolist() if 'review' in col]

['number_of_reviews',
 'first_review',
 'last_review',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'reviews_per_month']

In [38]:
df_test = df.dropna()
mod =smf.ols(formula='availability_365 ~ availability_90+number_of_reviews+review_scores_accuracy +review_scores_cleanliness+ review_scores_checkin+ review_scores_value+ reviews_per_month', data=df)


In [39]:
res = mod.fit()
print (res.summary())

                            OLS Regression Results                            
Dep. Variable:       availability_365   R-squared:                       0.452
Model:                            OLS   Adj. R-squared:                  0.452
Method:                 Least Squares   F-statistic:                     1864.
Date:                Wed, 12 Jul 2017   Prob (F-statistic):               0.00
Time:                        13:27:08   Log-Likelihood:                -94527.
No. Observations:               15809   AIC:                         1.891e+05
Df Residuals:                   15801   BIC:                         1.891e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             